# Banco de dados

No notebook passado aprendemos a trabalhar com o banco de dados SQLite3 muito utilizado para testes. Neste notebook trabalharemos com outros bancos de dados. Seu trabalho ao final dessa aula será fazer um CRUD em um banco de dados diferente do que veremos nessa aula e diferente do que vimos da aula passada. As sugestões de bancos de dados para essa tarefa se encontram no final do notebook. Seguiremos com o exemplo dessa aula que deve ser replicado para sua escolha.

## Escolha do banco

- O banco escolhido foi o POSTGRESQL
- Sistema operacional Windows
- Versão escolhida 17.5
- Download do instalador POSTGRESQL em https://www.postgresql.org/download/windows/
- Programa para conexão opcional Dbeaver Community 
- Download do instalador Dbeaver Community em https://dbeaver.io/download/

## Biblioteca escolhida para conexão

In [ ]:
# Instalação
# !pip install pg8000

In [1]:
# importação
import pg8000.native

## CONNECTION

### Com Dbeaver

<img src="assets/dbeaver.png" width="400px" height="300px">

In [3]:
# criaremos a conexão com nosso banco de dados
DB_HOST = "127.0.0.1" # Endereço do servidor de banco de dados
DB_NAME = "postgres" # Nome do banco de dados
DB_USER = "postgres" # Usuário do banco de dados
DB_PASSWORD = "1234" # Senha do banco de dados
DB_PORT = 5432 # Porta do banco de dados

In [ ]:
try:
    # Conectando ao banco de dados
    conn = pg8000.native.Connection(
        host=DB_HOST,
        database=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        port=DB_PORT
    )
    print("Tudo certo!")
except Exception as e:
    print("Erro:", e)
finally:
    if conn:
        conn.close()

Tudo certo!


# CREATE TABLE

In [ ]:
# Podemos agora criar tabelas 
# alguns bancos como o MongoDB não possuem o conceito de tabelas, mas sim coleções
# criaremos a tabela (table) cars com campos (fields) id, brand, model e year
# observe que a sintaxe é similar ao SQLite3 mas não é idêntica
# VARCHAR é usado para strings de tamanho variável, INT para inteiros
conn = pg8000.native.Connection(
    host=DB_HOST,
    database=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    port=DB_PORT
)
conn.run("""
    CREATE TABLE cars (
        brand VARCHAR(255),
        model VARCHAR(255),
        year INT
    );  
""")
conn.close()

Podemos conferir se a tabela foi realmente criada pelo Dbeaver.

<img src='assets/postgres_table_cars.png' width="300px" height="200px">

##  CREATE ROW

In [ ]:
# criaremos agora um registro (row) na tabela cars
# Novamente a nomencratura pode variar de banco para banco
conn = pg8000.native.Connection(
    host=DB_HOST,
    database=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    port=DB_PORT
)
conn.run("""
    INSERT INTO cars (brand, model, year)
    VALUES ('Ford', 'Mustang', 1964);
""")
conn.close()

In [9]:
# criaremos agora um segundo registro (row) na tabela cars
# Novamente a nomencratura pode variar de banco para banco
conn = pg8000.native.Connection(
    host=DB_HOST,
    database=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    port=DB_PORT
)
conn.run("""
    INSERT INTO cars (brand, model, year)
    VALUES ('Fiat', 'Uno', 1989);
""")
conn.close()

## READ ALL ROWS

In [10]:
# criaremos agora um registro (row) na tabela tasks
# Novamente a nomencratura pode variar de banco para banco
conn = pg8000.native.Connection(
    host=DB_HOST,
    database=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    port=DB_PORT
)
results = conn.run("""
    SELECT * FROM cars;
""")
for result in results:
    print(result)
conn.close()

['Ford', 'Mustang', 1964]
['Fiat', 'Uno', 1989]


# READ WITH FILTER

In [11]:
conn = pg8000.native.Connection(
    host=DB_HOST,
    database=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    port=DB_PORT
)
results = conn.run("""
    SELECT model, year 
    FROM cars
    WHERE year > 1980;
""")
for result in results:
    print(result)
conn.close()

['Uno', 1989]


# UPDATE

In [12]:
# Atualizar (update) na tabela tasks as linhas (rows) cujo model = UNO
# fazendo (SET) year = 1980
conn = pg8000.native.Connection(
    host=DB_HOST,
    database=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    port=DB_PORT
)
conn.run("""
    UPDATE cars
    SET year = 1980
    WHERE model = 'Uno';
""")
results = conn.run("""
    SELECT * FROM cars;
""")
for result in results:
    print(result)
conn.close()

['Ford', 'Mustang', 1964]
['Fiat', 'Uno', 1980]


# DELETE

In [13]:
# deletar da (FROM) tabela tasks as linhas (rows) onde id = 1
conn = pg8000.native.Connection(
    host=DB_HOST,
    database=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    port=DB_PORT
)
conn.run("""
    DELETE FROM cars
    WHERE model = 'Uno';
""")
results = conn.run("""
    SELECT * FROM cars;
""")
for result in results:
    print(result)
conn.close()

['Ford', 'Mustang', 1964]


## Outras bibliotecas para conexão

In [11]:
# https://www.psycopg.org/docs/
# pip install psycopg2

In [4]:
import psycopg2

try:
    # Conectando ao banco de dados
    conn = psycopg2.connect(
        f"dbname={DB_NAME} user={DB_USER} port={DB_PORT} host={DB_HOST} password={DB_PASSWORD}")
    print("Tudo certo!")
except Exception as e:
    print("Erro:", e)
finally:
    if conn:
        conn.close()

Tudo certo!


## Exemplo usando Psycopg2

In [10]:
import pandas as pd
conn = psycopg2.connect(
    f"dbname={DB_NAME} user={DB_USER} port={DB_PORT} host={DB_HOST} password={DB_PASSWORD}")
cursor = conn.cursor()
cursor.execute(
    "SELECT * FROM cars;"
)

results = cursor.fetchall()
for result in results:
    print(result)
conn.close()

('Ford', 'Mustang', 1964)


## ORM

AS duas bibliotecas que vimos anteriormentes são específicas para conexões com bancos de dados Postgresql. Um ORM é uma bibliteca que pode se conexar com bancos de mais de um tipo e suas consultas (query) são feitas utilizando a linguagem Python. Veremos um exemplo.

In [14]:
from sqlalchemy import create_engine, MetaData, Table, select

# Criando uma engine de conexão com o banco de dados PostgreSQL
engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")
# Criando a conexão
connection = engine.connect()
# Instanciando o MetaData
metadata = MetaData()
# Carregando a tabela 'cars' do banco de dados
cars = Table('cars', metadata, autoload_with=engine)
# criando uma consulta para selecionar todos os registros da tabela 'cars'
stmt = select(cars)
# Executando a consulta e obtendo os resultados
results = connection.execute(stmt)
for result in results:
    print(result)
connection.close()

('Ford', 'Mustang', 1964)


Acima vemos que não foi preciso usar SQL para criar uma consulta, uma vez que a consulta foi feita usando métodos e objetos da linguagem Python disponíveis na biblioteca sqlachemy. Mesmo não sendo o objetivo de uma ORM, é possível executar consultas em SQL diretamente.

In [18]:
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker

# 1. Cria a engine
engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

# 2. Cria a session
Session = sessionmaker(bind=engine)
session = Session()

# 3. Escreve a consulta SQL pura
sql = text("SELECT * FROM cars;")

# 4. Executa a consulta e pega os resultados
results = session.execute(sql)

# 5. Itera sobre os registros
for result in results:
    print(result)  # transforma em dicionário para facilitar o uso

# 6. Fecha a session
session.close()

('Ford', 'Mustang', 1964)


## Qual biblioteca usar?

Você já sabe a resposta, depende do objetivo. Se há possibilidade de mudar o banco de dados, por exemplo de um POSTGRESQL para um MYSQL, uma ORM é indicado. Para desenvolvimento de aplicações web com Django o psycopg2 é indicado. 

## Integração com pandas

Uma das vantagens do sqlalchemy é sua integração com a biblioteca pandas.

In [21]:
import pandas as pd
from sqlalchemy import create_engine

# Criando uma engine de conexão com o banco de dados PostgreSQL
engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

# Consulta SQL
query = "SELECT * FROM cars"

# Lê os dados diretamente em um DataFrame
df = pd.read_sql(query, engine)

# Fechando a conexão com o banco de dados
engine.dispose()

# Mostra os dados
df

,brand,model,year
0,Ford,Mustang,1964


Também é possível integrar pandas sem usar queries em SQL, usando apenas a sintaxe da ORM.

In [26]:
from sqlalchemy import select
import pandas as pd

from sqlalchemy import Table, MetaData

metadata = MetaData()
cars = Table("cars", metadata, autoload_with=engine)

stmt = select(cars)
df = pd.read_sql(stmt, engine)
df

,brand,model,year
0,Ford,Mustang,1964


## Exercício

Agora é sua vez. Você deve fazer um tutorial parecido com esse explorando novas bibliotecas de conexão com novos bancos de dados. Seu tutorial deve ter no mínimo:

- Conexão com o banco de dados
- Criação de tabela
- Criação de registros no banco de dados
- Leitura de registros sem filtro
- Leitura de registros com filtro
- Atualização de registro
- Deleção de registro
- Integração com Pandas

Algumas sugestões de bancos para utilizar:
- Redis
- MongoDB
- Mysql
- MariaDB